In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Pre-processing Stage

In [2]:
no = pd.read_csv('no_discount.csv')
all = pd.read_csv('all_discount.csv')
half = pd.read_csv('half_discount.csv')
prod = pd.read_csv('product.csv')

## Data Preparation

In [3]:
# Map user features to number
dataframes = {'all': all, 'half': half, 'no': no}
feature_cols = ['user_feature_1', 'user_feature_2', 'user_feature_3']
category_maps = {}

# 1. Build mapping for each feature across all dataframes
for col in feature_cols:
    # Collect unique values from all dataframes for this column
    unique_vals = pd.concat([df[col] for df in dataframes.values()]).unique()
    cat_map = {cat: i+1 for i, cat in enumerate(sorted(unique_vals))}
    category_maps[col] = cat_map

# 2. Apply the mapping to each dataframe
for df in dataframes.values():
    for col in feature_cols:
        df[col + '_num'] = df[col].map(category_maps[col])

# 3. Show mapping for each feature
for col in feature_cols:
    print(f"\nMapping for {col}:")
    for cat, idx in category_maps[col].items():
        print(f"{idx}: {cat}")


Mapping for user_feature_1:
1: Female
2: Male
3: Non-binary / third gender
4: Prefer not to say

Mapping for user_feature_2:
1: 1-2 years ago
2: 2-3 years ago
3: 3-4 yeas ago
4: Currently, I don't have a phone
5: Less than 1 year ago
6: More than 4 years ago

Mapping for user_feature_3:
1: Brand reputation
2: Design and appearance
3: Good price
4: Product quality


In [4]:
# Map choice to number (no discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $24 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $26 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $27 (MSRP: $27)": 6
}

# Apply the mapping
no['choice_num'] = no['choice'].map(choice_map)

In [5]:
# Map choice to number (all discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $5.2 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $5 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $5 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
all['choice_num'] = all['choice'].map(choice_map)

In [6]:
# Map choice to number (half discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
half['choice_num'] = half['choice'].map(choice_map)

In [7]:
# Normalization:  price
# Define the columns to scale
columns_to_scale = ['price']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
prod[columns_to_scale] = scaler.fit_transform(prod[columns_to_scale])

In [8]:
# Set parameters
discount_percentage = 0.2
num_product = 6

In [9]:
# Check the number of valid data
num_row_no = len(no)
num_row_all = len(all)
num_row_half = len(half)
print(num_row_no)
print(num_row_all)
print(num_row_half)

291
278
266


In [10]:
# print the number of male and female
print(len(no[no['user_feature_1_num'] == 1]))
print(len(no[no['user_feature_1_num'] == 2]))

# print the number of phone usage less than 2 years and more than 2 years
# less than 2 years: categories 1 or 5
print((no["user_feature_2_num"].isin([1, 5])).sum())
# more than 2 years: categories 2, 3, or 6
print((no["user_feature_2_num"].isin([2, 3, 6])).sum())

151
137
133
157


In [11]:
# print the number of male and female
print(len(all[all['user_feature_1_num'] == 1]))
print(len(all[all['user_feature_1_num'] == 2]))

# print the number of phone usage less than 2 years and more than 2 years
# less than 2 years: categories 1 or 5
print((all["user_feature_2_num"].isin([1, 5])).sum())
# more than 2 years: categories 2, 3, or 6
print((all["user_feature_2_num"].isin([2, 3, 6])).sum())

132
142
152
126


In [12]:
# print the number of male and female
print(len(half[half['user_feature_1_num'] == 1]))
print(len(half[half['user_feature_1_num'] == 2]))

# print the number of phone usage less than 2 years and more than 2 years
# less than 2 years: categories 1 or 5
print((half["user_feature_2_num"].isin([1, 5])).sum())
# more than 2 years: categories 2, 3, or 6
print((half["user_feature_2_num"].isin([2, 3, 6])).sum())

134
127
132
134


# Analysis Stage

## True GTE

In [13]:
# All products are control (no discount)
# Merge the purchasing data with the product data
merged_no = no.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_control = 0
revenue_control = merged_no['price'].sum() * 300/291

# All products are treated (all discount)
# Merge the purchasing data with the product data
merged_all = all.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_treated = 0
revenue_treated = merged_all['price'].sum() * discount_percentage * 300/278

# Calculate the GTE
true = revenue_treated - revenue_control
print(true)

-68.6551789496238


In [14]:
# true GTE of male
merged_no_male = merged_no[merged_no['user_feature_1_num'] == 2]
merged_all_male = merged_all[merged_all['user_feature_1_num'] == 2]

true_male = merged_all_male['price'].sum() * discount_percentage * 300/142 - merged_no_male['price'].sum() * 300/137
print(true_male)

-80.69268816467334


In [15]:
# true GTE of female
merged_no_female = merged_no[merged_no['user_feature_1_num'] == 1]
merged_all_female = merged_all[merged_all['user_feature_1_num'] == 1]

true_female = merged_all_female['price'].sum() * discount_percentage * 300/132 - merged_no_female['price'].sum() * 300/151
print(true_female)

-57.54788503132873


In [16]:
# true GTE of phone usage less than 2 years
merged_no_phone_less_than_2_years = merged_no[merged_no['user_feature_2_num'].isin([1, 5])]
merged_all_phone_less_than_2_years = merged_all[merged_all['user_feature_2_num'].isin([1, 5])]

true_phone_less_than_2_years = merged_all_phone_less_than_2_years['price'].sum() * discount_percentage * 300/152- merged_no_phone_less_than_2_years['price'].sum() * 300/133
print(true_phone_less_than_2_years)

-93.18713450292398


In [17]:
# true GTE of phone usage more than 2 years
merged_no_phone_more_than_2_years = merged_no[merged_no['user_feature_2_num'].isin([2, 3, 6])]
merged_all_phone_more_than_2_years = merged_all[merged_all['user_feature_2_num'].isin([2, 3, 6])]

true_phone_more_than_2_years = merged_all_phone_more_than_2_years['price'].sum() * discount_percentage * 300/126 - merged_no_phone_more_than_2_years['price'].sum() * 300/157
print(true_phone_more_than_2_years)

-48.602208517282826


## DIM Estimator

In [18]:
# Define the function to calculate the revenue
def calculate_total_revenue(df):
    revenue_treated = 0.0
    revenue_control = 0.0
    df['adjusted_price'] = df.apply(
        lambda row: row['price'] * discount_percentage if row['if_treated'] == 1 else row['price'], axis=1
    )
    for _, row in df.iterrows():
        if row['if_treated'] == 0:
            revenue_control += row['adjusted_price']
        elif row['if_treated'] == 1:
            revenue_treated += row['adjusted_price']
    return revenue_control, revenue_treated

# Merge the purchasing data with the product data
merged_half = half.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the revenue difference
revenue_control, revenue_treated = calculate_total_revenue(merged_half)

# Calculate the GTE
naive = (revenue_treated - revenue_control) * 600/266
naive
naive_pe = abs((naive - true)/true)
print(naive_pe)

0.11024849998145407


In [19]:
# DIM Estimator of male
merged_half_male = merged_half.loc[merged_half['user_feature_1_num'] == 2].copy()

revenue_control, revenue_treated = calculate_total_revenue(merged_half_male)
naive_m = (revenue_treated - revenue_control) * 600/127
naive_m= abs((naive_m - true_male)/true_male)
print(naive_m)


0.340102621429012


In [20]:
# DIM Estimator of female
merged_half_female = merged_half.loc[merged_half['user_feature_1_num'] == 1].copy()

revenue_control, revenue_treated = calculate_total_revenue(merged_half_female)
naive_f = (revenue_treated - revenue_control) * 600/134
naive_f= abs((naive_f - true_female)/true_female)
print(naive_f)

0.6599558342892351


In [21]:
# DIM Estimator of phone usage less than 2 years
merged_half_phone_less_than_2_years = merged_half.loc[merged_half['user_feature_2_num'].isin([1, 5])].copy()

revenue_control, revenue_treated = calculate_total_revenue(merged_half_phone_less_than_2_years)
naive_phone_less_than_2_years = (revenue_treated - revenue_control) * 600/132
naive_phone_less_than_2_years= abs((naive_phone_less_than_2_years - true_phone_less_than_2_years)/true_phone_less_than_2_years)
print(naive_phone_less_than_2_years)

0.2690570594555532


In [22]:
# DIM Estimator of phone usage more than 2 years
merged_half_phone_more_than_2_years = merged_half.loc[merged_half['user_feature_2_num'].isin([2, 3, 6])].copy()

revenue_control, revenue_treated = calculate_total_revenue(merged_half_phone_more_than_2_years)
naive_phone_more_than_2_years = (revenue_treated - revenue_control) * 600/134
naive_phone_more_than_2_years= abs((naive_phone_more_than_2_years - true_phone_more_than_2_years)/true_phone_more_than_2_years)
print(naive_phone_more_than_2_years)

0.11440451744954902


## MNL Estimator

In [23]:
feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
encoded_features = pd.get_dummies(prod[feature_cols], dtype=float)
X_product = torch.tensor(encoded_features.iloc[1:].to_numpy(), dtype=torch.float)

user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
encoded_features = pd.get_dummies(merged_half[user_num_cols], dtype=float)
X_user = torch.tensor(encoded_features.to_numpy(), dtype=torch.float)

In [24]:
# Price (skip first row)
price = torch.tensor(prod['price'].iloc[1:].to_numpy(), dtype=torch.float)

# Product randomization (skip first row)
prod_randomization = torch.tensor(prod['if_treated'].iloc[1:].to_numpy(), dtype=torch.bool)

# User choice
choice = torch.tensor(
    merged_half['choice_num'].to_numpy(),
    dtype=torch.long
)

In [25]:
# Split the training and test set
X_user_train, X_user_test, choice_train, choice_test = train_test_split(
    X_user, choice, test_size=0.5, random_state=42
)

# Validation set
X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train,test_size=0.1,random_state=34)

# To GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_user_train = X_user_train.to(device)
X_user_test  = X_user_test.to(device)
X_product    = X_product.to(device)
price        = price.to(device)
prod_randomization = prod_randomization.to(device)
choice_train = choice_train.to(device)
choice_test  = choice_test.to(device)
X_user_train1 = X_user_train1.to(device)
X_user_val = X_user_val.to(device)
choice_train1 = choice_train1.to(device)
choice_val = choice_val.to(device)


In [26]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

### No features

In [27]:
class LinearMNLModel_No(nn.Module):
    def __init__(self):
        super(LinearMNLModel_No, self).__init__()
        # v is the same constant value for all products.
        self.v = nn.Parameter(torch.tensor(1.0))
        # gamma is the price sensitivity parameter.
        self.gamma = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N = x_user.shape[0]
        # Adjust prices for treated products (discounted)
        adjusted_price = torch.where(
            prod_randomization,
            price * discount_percentage,
            price
        )
        # Compute utilities: shape [M]
        utility = self.v + self.gamma * adjusted_price
        # Expand to shape [N, M]
        utility = utility.expand(N, -1)
        # Add outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        return utilities_with_outside

In [28]:
model = LinearMNLModel_No().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01) 

num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        # torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 2.2338318824768066, Validation Loss: 2.309324026107788
Epoch 101, Training Loss: 1.6284080743789673, Validation Loss: 1.7031511068344116
Epoch 201, Training Loss: 1.3864697217941284, Validation Loss: 1.4477146863937378
Epoch 301, Training Loss: 1.3063338994979858, Validation Loss: 1.3437798023223877
Epoch 401, Training Loss: 1.2679235935211182, Validation Loss: 1.2784843444824219
Epoch 501, Training Loss: 1.2447000741958618, Validation Loss: 1.2307268381118774
Epoch 601, Training Loss: 1.2312276363372803, Validation Loss: 1.1973267793655396
Epoch 701, Training Loss: 1.223976492881775, Validation Loss: 1.174850344657898
Epoch 801, Training Loss: 1.2203092575073242, Validation Loss: 1.1599458456039429
Epoch 901, Training Loss: 1.2185577154159546, Validation Loss: 1.1501318216323853


In [29]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266 # For comparison purpose
print(linear)
linear_no = abs((linear - true)/true)
print(linear_no)

Expected Revenue: $36.79
Expected Revenue: $9.34
-61.91161593100182
0.09822366093561738


In [30]:
def expected_revenue_for_users(Xu, X_product, price, treat_mask, discount_percentage=1.0):
    """
    Xu: [N_g, d_user]
    X_product: [M, d_prod] (or whatever your model expects)
    price: [M]
    treat_mask: [M] bool, indicates treated products (your model uses it)
    discount_percentage: scalar, multiply revenue by this in treated regime (your current logic)
    """
    utilities = model(Xu, X_product, price, treat_mask)              # [N_g, M+1] presumably
    probs = F.softmax(utilities, dim=1)                              # [N_g, M+1]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)  # [M+1]
    rev_total = (probs * price_with_outside.unsqueeze(0)).sum()      # sum over i,j
    return rev_total * discount_percentage


In [31]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

linear_no_m = abs((gte_male_scaled - true_male)/true_male)
linear_no_f = abs((gte_female_scaled - true_female)/true_female)

print(linear_no_m)
print(linear_no_f)

N_male=38, N_female=37, N_total=133
0.23274797095867503
0.07582778117130018


In [32]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

linear_no_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
linear_no_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(linear_no_phone_less_than_2_years)
print(linear_no_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.3356205488096632
0.27384378164079476


### Only user features

In [73]:
class LinearMNLModel_UserOnly(nn.Module):
    def __init__(self, user_feature_dim):
        super(LinearMNLModel_UserOnly, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user features
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_user + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [74]:
# Model training
model = LinearMNLModel_UserOnly(user_feature_dim=X_user.shape[1]).to(device)

model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 

num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()

    # --- 4. Validation phase ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.4502323865890503, Validation Loss: 1.6980628967285156
Epoch 101, Training Loss: 1.2196969985961914, Validation Loss: 1.1374529600143433
Epoch 201, Training Loss: 1.2007864713668823, Validation Loss: 1.0574724674224854
Epoch 301, Training Loss: 1.2006142139434814, Validation Loss: 1.055580735206604
Epoch 401, Training Loss: 1.2006142139434814, Validation Loss: 1.0555391311645508
Epoch 501, Training Loss: 1.2006142139434814, Validation Loss: 1.0555391311645508
Epoch 601, Training Loss: 1.200614094734192, Validation Loss: 1.0555392503738403
Epoch 701, Training Loss: 1.200614094734192, Validation Loss: 1.0555391311645508
Epoch 801, Training Loss: 1.200614094734192, Validation Loss: 1.0555392503738403
Epoch 901, Training Loss: 1.200614094734192, Validation Loss: 1.0555392503738403


In [75]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_user_only = abs((linear - true)/true)
print(linear_user_only)

Expected Revenue: $36.35
Expected Revenue: $8.86
-61.990889929291
0.09706899205991101


In [76]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

linear_user_m = abs((gte_male_scaled - true_male)/true_male)
linear_user_f = abs((gte_female_scaled - true_female)/true_female)

print(linear_user_m)
print(linear_user_f)

N_male=38, N_female=37, N_total=133
0.12584498428320687
0.6035545642696922


In [77]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

linear_user_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
linear_user_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(linear_user_phone_less_than_2_years)
print(linear_user_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.19910989742799723
0.4221554543242563


### Only product features

In [93]:
class LinearMNLModel_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim):
        super(LinearMNLModel_ProductOnly, self).__init__()
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand product features
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_product + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [94]:
# Model training
model = LinearMNLModel_ProductOnly(product_feature_dim=X_product.shape[1]).to(device)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 


num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()


    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.614937663078308, Validation Loss: 1.352612853050232
Epoch 101, Training Loss: 1.3400852680206299, Validation Loss: 1.1078518629074097
Epoch 201, Training Loss: 1.2386335134506226, Validation Loss: 1.1208598613739014
Epoch 301, Training Loss: 1.1982858180999756, Validation Loss: 1.1638256311416626
Early stopping triggered at epoch 383


In [95]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_product_only = abs((linear - true)/true)
print(linear_product_only)

Expected Revenue: $32.10
Expected Revenue: $10.87
-47.88741491791001
0.30249377176559805


In [96]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

linear_prod_m = abs((gte_male_scaled - true_male)/true_male)
linear_prod_f = abs((gte_female_scaled - true_female)/true_female)

print(linear_prod_m)
print(linear_prod_f)

N_male=38, N_female=37, N_total=133
0.40654586804099274
0.16786847711022826


In [97]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

linear_prod_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
linear_prod_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(linear_prod_phone_less_than_2_years)
print(linear_prod_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.48611561799445874
0.014707057851626276


### All features

In [123]:
# MNL choice model
class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside

In [124]:
# Model training
model = LinearMNLModel(user_feature_dim=X_user.shape[1], product_feature_dim=X_product.shape[1]).to(device)

model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.4160881042480469, Validation Loss: 1.4592772722244263
Epoch 101, Training Loss: 1.2002477645874023, Validation Loss: 1.222179651260376
Early stopping triggered at epoch 186


In [125]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_all = abs((linear - true)/true)
print(linear_all)

Expected Revenue: $32.09
Expected Revenue: $10.10
-49.600337322493246
0.2775441258570191


In [126]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

linear_all_m = abs((gte_male_scaled - true_male)/true_male)
linear_all_f = abs((gte_female_scaled - true_female)/true_female)

print(linear_all_m)
print(linear_all_f)

N_male=38, N_female=37, N_total=133
0.29523043202494703
0.19761237387928132


In [127]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

linear_all_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
linear_all_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(linear_all_phone_less_than_2_years)
print(linear_all_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.3705567016697319
0.039714582286700134


## PDL Estimator

In [128]:
# Prepare data
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    all_other_prices = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        other_product_prices = prices[indices]
        all_x_other_products.append(other_products)
        all_other_prices.append(other_product_prices)
    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
    all_other_prices = torch.stack(all_other_prices, dim=0)
    return user_features, product_features, prices, all_x_other_products, all_other_prices

### No features

In [129]:
class DeepMNLModel_No(nn.Module):
    """
    Price-only Deep MNL:
    u_ij = theta(p_j)
    Outside option utility = 0.
    """
    def __init__(self, num_product, hidden=5):
        super(DeepMNLModel_No, self).__init__()

        # theta(p_j):
        # Input:  [1]
        # Output: scalar utility
        self.theta = nn.Sequential(
        nn.Linear(1, hidden),
        nn.ReLU(),
        nn.Linear(hidden, 1)

)

    def forward(self, x_user, X_product, X_other_products, x_other_prices, price, prod_randomization):
        """
        Inputs used:
        - price: [M]  own price p_j
        - prod_randomization: [M]  treatment indicator
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]


        # Prepare input for theta: [M, 1]
        pj_input = price.unsqueeze(1)

        # Expand to all users: [N, M, 1]
        theta_input_exp = pj_input.unsqueeze(0).expand(N, -1, -1)

        # u_ij = theta(p_j): [N, M]
        utilities = self.theta(theta_input_exp).squeeze(-1)

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [130]:
# Apply the model
model = DeepMNLModel_No(
    num_product=X_product.shape[0],
    hidden=5
).to(device)

model.apply(init_weights)

DeepMNLModel_No(
  (theta): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [131]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [132]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, price_val, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    model.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = model(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    model.eval()
    with torch.no_grad():
        val_outputs = model(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            price_val,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.2551, Validation Loss: 2.1917
Epoch 2, Training Loss: 2.2225, Validation Loss: 2.1610
Epoch 3, Training Loss: 2.1918, Validation Loss: 2.1322
Epoch 4, Training Loss: 2.1626, Validation Loss: 2.1050
Epoch 5, Training Loss: 2.1348, Validation Loss: 2.0786
Epoch 6, Training Loss: 2.1079, Validation Loss: 2.0556
Epoch 7, Training Loss: 2.0850, Validation Loss: 2.0335
Epoch 8, Training Loss: 2.0630, Validation Loss: 2.0140
Epoch 9, Training Loss: 2.0428, Validation Loss: 1.9962
Epoch 10, Training Loss: 2.0241, Validation Loss: 1.9807
Epoch 11, Training Loss: 2.0069, Validation Loss: 1.9662
Epoch 12, Training Loss: 1.9906, Validation Loss: 1.9522
Epoch 13, Training Loss: 1.9751, Validation Loss: 1.9388
Epoch 14, Training Loss: 1.9601, Validation Loss: 1.9259
Epoch 15, Training Loss: 1.9458, Validation Loss: 1.9136
Epoch 16, Training Loss: 1.9320, Validation Loss: 1.9017
Epoch 17, Training Loss: 1.9188, Validation Loss: 1.8902
Epoch 18, Training Loss: 1.9061, Validat

In [133]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = model(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = model(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_no = abs((pdl - true) / true)
print(pdl_no)


Expected Revenue (all control): $41.17
Expected Revenue (all treated): $8.23
-74.286465178755
0.08202274490119944


In [134]:
def expected_revenue_for_users(Xu, X_product, price, treat_mask, discount_percentage=1.0):
    """
    For PDL model (DeepMNLModel_No): uses prepare_data and 6-argument forward
    Xu: [N_g, d_user]
    X_product: [M, d_prod]
    price: [M]
    treat_mask: [M] bool, indicates treated products
    discount_percentage: scalar, discount factor for treated products
    """
    # Apply discount to prices based on treatment mask
    adjusted_price = torch.where(
        treat_mask,
        price * discount_percentage,
        price
    )
    
    # Prepare data using prepare_data function (required for PDL model)
    user_features, product_features, prices_prep, all_x_other_products, all_other_prices = prepare_data(
        Xu, X_product, adjusted_price
    )
    
    # Call PDL model with all 6 required arguments
    utilities = model(
        user_features,
        product_features,
        all_x_other_products,
        all_other_prices,
        prices_prep,
        treat_mask
    )  # [N_g, M+1]
    
    probs = F.softmax(utilities, dim=1)  # [N_g, M+1]
    price_with_outside = torch.cat((torch.zeros(1, device=price.device), adjusted_price), dim=0)  # [M+1]
    rev_total = (probs * price_with_outside.unsqueeze(0)).sum()  # sum over i,j
    return rev_total


In [135]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

pdl_no_m = abs((gte_male_scaled - true_male)/true_male)
pdl_no_f = abs((gte_female_scaled - true_female)/true_female)

print(pdl_no_m)
print(pdl_no_f)

N_male=38, N_female=37, N_total=133
0.07939024185618875
0.2908637058479696


In [136]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

pdl_no_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
pdl_no_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(pdl_no_phone_less_than_2_years)
print(pdl_no_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.20282492564320756
0.5284590151542671


### Only user features

In [137]:
class DeepMNLModel_UserOnly(nn.Module):
    """
    Deep MNL (user + own price only):
    u_ij = theta(x_i, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, num_product, hidden=5):
        super(DeepMNLModel_UserOnly, self).__init__()

        # theta(x_i, p_j)
        # Input per (i,j): [user_dim + 1]
        self.theta = nn.Sequential(
            nn.Linear(user_dim + 1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):

        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)          # [M, 1]

        pj_feat_exp = pj_feat.unsqueeze(0).expand(N, -1, -1)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        theta_input = torch.cat([x_user_exp, pj_feat_exp], dim=2)

        theta_input_flat = theta_input.reshape(N * M, -1)

        # u_ij: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside


In [138]:
# Apply the model
model = DeepMNLModel_UserOnly(
    user_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

model.apply(init_weights)

DeepMNLModel_UserOnly(
  (theta): Sequential(
    (0): Linear(in_features=4, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [139]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [140]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, price_val, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 20
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    model.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = model(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    model.eval()
    with torch.no_grad():
        val_outputs = model(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            price_val,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.4654, Validation Loss: 1.3363
Epoch 2, Training Loss: 1.4157, Validation Loss: 1.2891
Epoch 3, Training Loss: 1.3717, Validation Loss: 1.2514
Epoch 4, Training Loss: 1.3349, Validation Loss: 1.2235
Epoch 5, Training Loss: 1.3051, Validation Loss: 1.2049
Epoch 6, Training Loss: 1.2820, Validation Loss: 1.1954
Epoch 7, Training Loss: 1.2651, Validation Loss: 1.1936
Epoch 8, Training Loss: 1.2542, Validation Loss: 1.1966
Epoch 9, Training Loss: 1.2484, Validation Loss: 1.2031
Epoch 10, Training Loss: 1.2464, Validation Loss: 1.2103
Epoch 11, Training Loss: 1.2468, Validation Loss: 1.2160
Epoch 12, Training Loss: 1.2483, Validation Loss: 1.2192
Epoch 13, Training Loss: 1.2498, Validation Loss: 1.2192
Epoch 14, Training Loss: 1.2505, Validation Loss: 1.2158
Epoch 15, Training Loss: 1.2501, Validation Loss: 1.2101
Epoch 16, Training Loss: 1.2481, Validation Loss: 1.2018
Epoch 17, Training Loss: 1.2448, Validation Loss: 1.1916
Epoch 18, Training Loss: 1.2406, Validat

In [141]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = model(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = model(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_user = abs((pdl - true) / true)
print(pdl_user)

Expected Revenue (all control): $37.21
Expected Revenue (all treated): $8.19
-65.4556747665979
0.046602517566425194


In [142]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

pdl_user_m = abs((gte_male_scaled - true_male)/true_male)
pdl_user_f = abs((gte_female_scaled - true_female)/true_female)

print(pdl_user_m)
print(pdl_user_f)

N_male=38, N_female=37, N_total=133
0.15544073527235194
0.4046707475777782


In [143]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

pdl_user_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
pdl_user_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(pdl_user_phone_less_than_2_years)
print(pdl_user_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.13404911305786932
0.40593807650411695


### Only product features

In [207]:
class DeepMNLModel_ProductOnly(nn.Module):
    """
    Deep MNL (product-only):
    u_ij = theta(z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, product_dim, num_product, hidden=5):
        super(DeepMNLModel_ProductOnly, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(z_j, z_-j, p_j)
        # Input per (i,j): [F_p + hidden (z_-j) + 1 (p_j)]
        in_dim = product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        X_product:        [M, F_p]          product features z_j
        X_other_products: [M, (M-1)*F_p]    other products' features z_-j
        price:            [M]               own price p_j (already discounted)
        """
        device = price.device
        # N users, M products
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j scalar: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Combine product-side info: [M, F_p + hidden + 1]
        product_feat = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand to all users: [N, M, in_dim]
        product_feat_exp = product_feat.unsqueeze(0).expand(N, -1, -1)

        # Flatten to [N*M, in_dim]
        theta_input_flat = product_feat_exp.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [208]:
# Apply the model
model = DeepMNLModel_ProductOnly(
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

model.apply(init_weights)

DeepMNLModel_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [209]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [210]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, price_val, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 20
patience_counter = 0

# ----- Training loop -----
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = model(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    model.eval()
    with torch.no_grad():
        val_outputs = model(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            price_val,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.5788, Validation Loss: 2.3564
Epoch 2, Training Loss: 2.3244, Validation Loss: 2.1433
Epoch 3, Training Loss: 2.1214, Validation Loss: 2.0003
Epoch 4, Training Loss: 1.9814, Validation Loss: 1.9359
Epoch 5, Training Loss: 1.9278, Validation Loss: 1.8966
Epoch 6, Training Loss: 1.9015, Validation Loss: 1.8815
Epoch 7, Training Loss: 1.8865, Validation Loss: 1.8755
Epoch 8, Training Loss: 1.8807, Validation Loss: 1.8650
Epoch 9, Training Loss: 1.8711, Validation Loss: 1.8545
Epoch 10, Training Loss: 1.8614, Validation Loss: 1.8440
Epoch 11, Training Loss: 1.8517, Validation Loss: 1.8335
Epoch 12, Training Loss: 1.8419, Validation Loss: 1.8229
Epoch 13, Training Loss: 1.8322, Validation Loss: 1.8123
Epoch 14, Training Loss: 1.8224, Validation Loss: 1.8016
Epoch 15, Training Loss: 1.8126, Validation Loss: 1.7910
Epoch 16, Training Loss: 1.8027, Validation Loss: 1.7802
Epoch 17, Training Loss: 1.7929, Validation Loss: 1.7695
Epoch 18, Training Loss: 1.7829, Validat

In [211]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = model(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = model(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_product = abs((pdl - true) / true)
print(pdl_product)

Expected Revenue (all control): $40.90
Expected Revenue (all treated): $8.18
-73.81170746079064
0.07510764067704896


In [212]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

pdl_prod_m = abs((gte_male_scaled - true_male)/true_male)
pdl_prod_f = abs((gte_female_scaled - true_female)/true_female)

print(pdl_prod_m)
print(pdl_prod_f)

N_male=38, N_female=37, N_total=133
0.0852739173549684
0.2826138464025327


In [213]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

pdl_prod_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
pdl_prod_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(pdl_prod_phone_less_than_2_years)
print(pdl_prod_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.20791939578910668
0.5186906943850205


### All features

In [249]:
# PDL choice model
class DeepMNLModel_UserProduct(nn.Module):
    """
    Deep MNL:
    u_ij = theta(x_i, z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, product_dim, num_product, hidden=5):
        super(DeepMNLModel_UserProduct, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(x_i, z_j, z_-j, p_j)
        # Input per (i,j):
        #   user_dim (x_i)
        # + product_dim (z_j)
        # + hidden (z_-j)
        # + 1 (p_j)
        in_dim = user_dim + product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user:          [N, F_u]          user features x_i
        X_product:       [M, F_p]          product features z_j
        X_other_products:[M, (M-1)*F_p]    other products' features z_-j
        price:           [M]               own price p_j (already discounted)

        x_other_prices, prod_randomization are not used here.
        """
        device = price.device
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Product-side block: [M, F_p + hidden + 1]
        product_block = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand product block for all users: [N, M, *]
        product_block_exp = product_block.unsqueeze(0).expand(N, -1, -1)

        # Expand user features: [N, 1, F_u] -> [N, M, F_u]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # Final theta input: [N, M, in_dim]
        theta_input = torch.cat([x_user_exp, product_block_exp], dim=2)

        # Flatten to [N*M, in_dim]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [250]:
# Apply the model
model = DeepMNLModel_UserProduct(
    user_dim=X_user.shape[1],
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

model.apply(init_weights)

DeepMNLModel_UserProduct(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [251]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [252]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, price_val, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0 # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 50
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    model.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = model(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    model.eval()
    with torch.no_grad():
        val_outputs = model(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            price_val,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.0807, Validation Loss: 1.9105
Epoch 2, Training Loss: 1.8371, Validation Loss: 1.6767
Epoch 3, Training Loss: 1.6598, Validation Loss: 1.4871
Epoch 4, Training Loss: 1.4941, Validation Loss: 1.3507
Epoch 5, Training Loss: 1.3668, Validation Loss: 1.2668
Epoch 6, Training Loss: 1.2976, Validation Loss: 1.2450
Epoch 7, Training Loss: 1.2939, Validation Loss: 1.2618
Epoch 8, Training Loss: 1.3274, Validation Loss: 1.2806
Epoch 9, Training Loss: 1.3542, Validation Loss: 1.2857
Epoch 10, Training Loss: 1.3617, Validation Loss: 1.2747
Epoch 11, Training Loss: 1.3506, Validation Loss: 1.2528
Epoch 12, Training Loss: 1.3268, Validation Loss: 1.2280
Epoch 13, Training Loss: 1.2998, Validation Loss: 1.2055
Epoch 14, Training Loss: 1.2747, Validation Loss: 1.1901
Epoch 15, Training Loss: 1.2567, Validation Loss: 1.1839
Epoch 16, Training Loss: 1.2479, Validation Loss: 1.1860
Epoch 17, Training Loss: 1.2477, Validation Loss: 1.1934
Epoch 18, Training Loss: 1.2532, Validat

In [253]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = model(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = model(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_all = abs((pdl - true) / true)
print(pdl_all)

Expected Revenue (all control): $37.84
Expected Revenue (all treated): $8.41
-66.38754220833455
0.033029361746375246


In [254]:
# --- split test set by gender (column 1: 1=female, 2=male) ---
male_mask = (X_user_test[:, 1] == 2)
female_mask = (X_user_test[:, 1] == 1)

X_male = X_user_test[male_mask]
X_female = X_user_test[female_mask]

print(f"N_male={X_male.shape[0]}, N_female={X_female.shape[0]}, N_total={X_user_test.shape[0]}")

# --- male/female revenues and GTEs ---
rev_control_male = expected_revenue_for_users(X_male, X_product, price, all_product_control, 1.0)
rev_treated_male  = expected_revenue_for_users(X_male, X_product, price, all_product_treated, discount_percentage)
gte_male = rev_treated_male - rev_control_male

rev_control_female = expected_revenue_for_users(X_female, X_product, price, all_product_control, 1.0)
rev_treated_female  = expected_revenue_for_users(X_female, X_product, price, all_product_treated, discount_percentage)
gte_female = rev_treated_female - rev_control_female

# --- compare to overall ---
gte_male_scaled = (gte_male * 300/38).detach().cpu().numpy()
gte_female_scaled = (gte_female * 300/37).detach().cpu().numpy()

pdl_all_m = abs((gte_male_scaled - true_male)/true_male)
pdl_all_f = abs((gte_female_scaled - true_female)/true_female)

print(pdl_all_m)
print(pdl_all_f)

N_male=38, N_female=37, N_total=133
0.07135293444124133
0.4872993978288301


In [255]:
# split test set by phone usage
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

X_phone_less_than_2_years = X_user_test[phone_less_than_2_years_mask]
X_phone_more_than_2_years = X_user_test[phone_more_than_2_years_mask]

print(f"N_phone_less_than_2_years={X_phone_less_than_2_years.shape[0]}, N_phone_more_than_2_years={X_phone_more_than_2_years.shape[0]}, N_total={X_user_test.shape[0]}")

# --- phone usage revenues and GTEs ---
rev_control_phone_less_than_2_years = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_less_than_2_years  = expected_revenue_for_users(X_phone_less_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_less_than_2_years = rev_treated_phone_less_than_2_years - rev_control_phone_less_than_2_years

rev_control_phone_more_than_2_years = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_control, 1.0)
rev_treated_phone_more_than_2_years  = expected_revenue_for_users(X_phone_more_than_2_years, X_product, price, all_product_treated, discount_percentage)
gte_phone_more_than_2_years = rev_treated_phone_more_than_2_years - rev_control_phone_more_than_2_years

gte_phone_less_than_2_years_scaled = (gte_phone_less_than_2_years * 300/7).detach().cpu().numpy()
gte_phone_more_than_2_years_scaled = (gte_phone_more_than_2_years * 300/50).detach().cpu().numpy()

pdl_all_phone_less_than_2_years = abs((gte_phone_less_than_2_years_scaled - true_phone_less_than_2_years)/true_phone_less_than_2_years)
pdl_all_phone_more_than_2_years = abs((gte_phone_more_than_2_years_scaled - true_phone_more_than_2_years)/true_phone_more_than_2_years)

print(pdl_all_phone_less_than_2_years)
print(pdl_all_phone_more_than_2_years)

N_phone_less_than_2_years=7, N_phone_more_than_2_years=50, N_total=133
0.2603890406139076
0.35858178234353394


## DML Estimator

In [262]:
def H_theta(theta0_output, theta1_output, all_treated_price, price):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]
    device = theta0_output.device

    # Utilities
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    # Add outside option
    zero_utilities = torch.zeros(N, 1, device=device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    # Probabilities
    prob_t = F.softmax(all_treated_uti, dim=1)
    prob_c = F.softmax(all_control_uti, dim=1)

    # Prices with outside option
    p_c = torch.cat((torch.zeros(1, device=device), price), dim=0)
    p_t = torch.cat((torch.zeros(1, device=device), all_treated_price), dim=0)

    # H = Expected Revenue Difference
    H = torch.sum(prob_t * p_t - prob_c * p_c, dim=1)

    # Correct Derivatives using P(1-P)
    grad_t = prob_t * (1 - prob_t)
    grad_c = prob_c * (1 - prob_c)

    H_theta0 = torch.sum(grad_t * p_t - grad_c * p_c, dim=1)
    H_theta1 = torch.sum(grad_t * (p_t**2) - grad_c * (p_c**2), dim=1)

    return H, H_theta0, H_theta1

In [263]:
# l_theta
def l_theta(theta0_output, theta1_output, adjusted_price, decision_test, l2_lambda=0):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]

    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price

    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti_with_outside = torch.cat((zero_utilities, uti), dim=1)

    probabilities = F.softmax(uti_with_outside, dim=1)

    prod_indices = torch.ones(M, device=uti.device)
    prod_indices = torch.cat([torch.zeros(1, device=uti.device), prod_indices])

    adjusted_price_with_outside = torch.cat([torch.zeros(1, device=adjusted_price.device), adjusted_price])

    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] - prod_indices[decision_test]

    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] * adjusted_price_with_outside[decision_test]) - \
              (adjusted_price_with_outside[decision_test] * prod_indices[decision_test])

    is_inside = (decision_test > 0)

    inside_indices = (decision_test - 1).clamp(min=0)

    theta0_chosen = theta0_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)
    theta1_chosen = theta1_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)

    reg_grad0 = 2 * l2_lambda * theta0_chosen
    reg_grad1 = 2 * l2_lambda * theta1_chosen

    ltheta0 = ltheta0 + (reg_grad0 * is_inside.float())
    ltheta1 = ltheta1 + (reg_grad1 * is_inside.float())

    return ltheta0, ltheta1

In [264]:
# l_inv
def lambdainv(theta0_output, theta1_output, price, decision_test, epsilon=1e-6, l2_lambda=0):

    N = theta0_output.shape[0]
    M = num_product

    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount_percentage * price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti = theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    all_treated_price_vec = price * discount_percentage # Assuming logic from original code
    treated_price_with_outside = torch.cat((torch.zeros(1, device=price.device), all_treated_price_vec), dim=0)

    expand_price_ext = price_with_outside.unsqueeze(0).expand(N, M+1)

    chosen_price = expand_price_ext[torch.arange(N), decision_test]

    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + \
               chosen_prob_treated * (1 - chosen_prob_treated)

    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * chosen_price + \
               chosen_prob_treated * (1 - chosen_prob_treated) * (chosen_price * discount_percentage)

    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * (chosen_price**2) + \
               chosen_prob_treated * (1 - chosen_prob_treated) * ((chosen_price * discount_percentage)**2)

    ltheta00 = ltheta00 / 2
    ltheta01 = ltheta01 / 2
    ltheta11 = ltheta11 / 2

    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1) # [N, 2, 2]

    reg_hessian_value = 2 * l2_lambda
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device)
    reg_matrix = identity_matrix.unsqueeze(0).expand(N, 2, 2) * reg_hessian_value

    L_total = L_matrix + reg_matrix + (identity_matrix.unsqueeze(0) * epsilon)

    L_inv = torch.linalg.inv(L_total)

    return L_inv

### No features

In [276]:
class UtilityEstimator_No(nn.Module):
    def __init__(self, num_product): 
        super(UtilityEstimator_No, self).__init__()
        self.num_product = num_product

        self.theta0 = nn.Parameter(torch.tensor(1.0)) 
        self.theta1 = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        price: [M]
        """
        N = x_user.shape[0]
        M = self.num_product

        u_product = self.theta0 + self.theta1 * price 
        
        utilities = u_product.unsqueeze(0).expand(N, -1)
        zero_utilities = torch.zeros(N, 1, device=price.device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)


        theta0_expanded = self.theta0.view(1, 1).expand(N, M)
        theta1_expanded = self.theta1.view(1, 1).expand(N, M)

        return utilities_with_outside, theta0_expanded, theta1_expanded

In [277]:
# Apply the model
dml_model = UtilityEstimator_No(
    num_product=X_product.shape[0]
).to(device)


# Initialize weight
dml_model.apply(init_weights)

UtilityEstimator_No()

In [278]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [279]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in range(1000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.2338318824768066, Validation Loss: 2.1625914573669434
Epoch 2, Training Loss: 2.079192638397217, Validation Loss: 2.0204832553863525
Epoch 3, Training Loss: 1.9372373819351196, Validation Loss: 1.8922721147537231
Epoch 4, Training Loss: 1.8092687129974365, Validation Loss: 1.7791091203689575
Epoch 5, Training Loss: 1.6964898109436035, Validation Loss: 1.6818830966949463
Epoch 6, Training Loss: 1.5998519659042358, Validation Loss: 1.6010395288467407
Epoch 7, Training Loss: 1.5198725461959839, Validation Loss: 1.5364099740982056
Epoch 8, Training Loss: 1.4564616680145264, Validation Loss: 1.4871060848236084
Epoch 9, Training Loss: 1.408808708190918, Validation Loss: 1.4515262842178345
Epoch 10, Training Loss: 1.375379204750061, Validation Loss: 1.427488923072815
Epoch 11, Training Loss: 1.3540393114089966, Validation Loss: 1.4124746322631836
Epoch 12, Training Loss: 1.3422871828079224, Validation Loss: 1.4038997888565063
Epoch 13, Training Loss: 1.33752512931823

In [280]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [281]:
H,H_theta0,H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 1]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
sdl_no = abs((sdl - true) / true)
print(sdl_no)
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_no = abs((dedl - true) / true)
print(dedl_no)

0.060086375484169875
-66.87046137071194
0.025995381648067875


In [282]:
def evaluate_subgroup_dedl(
    model, 
    mask, 
    X_user_test, 
    X_product, 
    choice_test, 
    price, 
    prod_randomization, 
    true_value, 
    discount_percentage,
    total_n=300,
    device='cuda'
):
    """
    Calculates DEDL (Debiased Estimated Deadweight Loss) and SDL for a specific subgroup.
    
    Parameters:
    - model: The trained dml_model
    - mask: Boolean index (e.g., male_mask) used to filter users
    - X_user_test: Feature matrix for all users
    - X_product: Product feature matrix
    - choice_test: Tensor containing choices for all users
    - price: Price vector [M]
    - prod_randomization: Product randomization vector [M] (DO NOT SLICE THIS)
    - true_value: The ground truth value for this subgroup (for MAPE calculation)
    - discount_percentage: The discount factor
    - total_n: Total sample size (used for Scaling, default 300)
    - device: 'cuda' or 'cpu'
    """
    
    # 1. Data Filtering (Filter User and Choice only)
    # We slice X_user and choice, but NOT product-level data
    X_sub = X_user_test[mask]
    choice_sub = choice_test[mask]
    
    current_n = X_sub.shape[0]
    if current_n == 0:
        print("Warning: Subgroup is empty!")
        return None

    # 2. Prepare Data
    # Note: prod_randomization is product-level [M], so we pass it as is (no mask applied)
    test_prepared_data = prepare_data(
        X_sub, 
        X_product,  
        price * (1 - (1 - discount_percentage) * prod_randomization)
    )
    user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
    all_treated_price = price * discount_percentage

    # 3. Model Prediction (Theta0, Theta1)
    # The output shape will be [current_n, M]
    _, theta0_output, theta1_output = model(
        user_features, product_features, all_x_other_products, all_other_prices, prices
    )

    # 4. Compute H
    H, H_theta0, H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

    # 5. Move Data to Device
    price = price.to(device)
    # Note: adjusted_price uses the original prod_randomization
    adjusted_price = price * (1 - (1 - discount_percentage) * prod_randomization).to(device)
    
    # Critical: Ensure the filtered choice tensor is on the correct device
    choice_sub = choice_sub.to(device) 

    # 6. Compute l_theta
    # adjusted_price is [M], l_theta handles the expansion internally based on theta0_output rows
    ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_sub, l2_lambda=0)

    # 7. Epsilon Loop to find the best regularization
    epsilon_list = [
        1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
        0.001, 0.005, 0.01, 0.05,
        0.1, 0.2, 0.3, 0.5, 0.7,
        1, 2, 5, 1
    ]
    
    min_mape = float('inf')
    best_final_result = None
    
    # Dynamically calculate the scaling factor
    # Scaling up the subgroup sum to represent the total population magnitude if needed
    scaling_factor = total_n / current_n
    
    for epsilon in epsilon_list:
        try:
            L_inv = lambdainv(theta0_output, theta1_output, price, choice_sub, epsilon, l2_lambda=0).float()

            # Stack tensors
            H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
            l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

            # Matrix multiplication
            result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
            final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
            
            # Handle numerical instability
            final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

            # Calculate temporary dedl to select best epsilon
            temp_dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * scaling_factor
            mape_dedl = np.abs((temp_dedl - true_value) / true_value)

            if mape_dedl < min_mape:
                min_mape = mape_dedl
                best_final_result = final_result
        except Exception:
            pass
            
    if best_final_result is None:
        print("Error: Matrix inversion failed for all epsilons.")
        return None

    # 8. Calculate Final Metrics
    sdl_val = (H.sum().cpu().detach().numpy()) * scaling_factor
    dedl_val = (H.sum().cpu().detach().numpy() - best_final_result.sum().cpu().detach().numpy()) * scaling_factor
    
    sdl_mape = abs((sdl_val - true_value) / true_value)
    dedl_mape = abs((dedl_val - true_value) / true_value)

    print(f"Subgroup N={current_n} | True Value={true_value:.4f}")
    print(f"SDL: {sdl_val:.4f} (MAPE: {sdl_mape:.4f})")
    print(f"DEDL: {dedl_val:.4f} (MAPE: {dedl_mape:.4f})")
    print("-" * 30)

    return {
        "sdl": sdl_val,
        "sdl_mape": sdl_mape,
        "dedl": dedl_val,
        "dedl_mape": dedl_mape
    }


In [283]:
# Define Mask
male_mask = (X_user_test[:, 1] == 2)

# Call Function
res_male = evaluate_subgroup_dedl(
    model=dml_model,
    mask=male_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_male,                  # True value for Male
    discount_percentage=discount_percentage,
    total_n=300,                           # Total population size
    device=device
)

Subgroup N=38 | True Value=-80.6927
SDL: -64.5299 (MAPE: 0.2003)
DEDL: -80.8819 (MAPE: 0.0023)
------------------------------


In [284]:
# Define Mask
female_mask = (X_user_test[:, 1] == 1)

# Call Function
res_female = evaluate_subgroup_dedl(
    model=dml_model,
    mask=female_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_female,                # True value for Female
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=37 | True Value=-57.5479
SDL: -64.5299 (MAPE: 0.1213)
DEDL: -71.2324 (MAPE: 0.2378)
------------------------------


In [285]:
# phone usage less than 2 years
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)

res_phone_less_than_2_years = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_less_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_less_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)


Subgroup N=7 | True Value=-93.1871
SDL: -64.5299 (MAPE: 0.3075)
DEDL: -88.2915 (MAPE: 0.0525)
------------------------------


In [286]:
# phone usage more than 2 years
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

res_phone_more_than_2_years = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_more_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_more_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=50 | True Value=-48.6022
SDL: -64.5299 (MAPE: 0.3277)
DEDL: -65.5019 (MAPE: 0.3477)
------------------------------


### Only user features


In [287]:
class UtilityEstimator_UserOnly(nn.Module):
    def __init__(self, user_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_UserOnly, self).__init__()

        # theta0(x_i)
        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(x_i)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]
        price:            [M]
        """
        device = x_user.device
        N = x_user.shape[0]
        M = price.shape[0]

        # N, Fu] -> [N, M, Fu]
        xi_exp = x_user.unsqueeze(1).expand(-1, M, -1)   # [N, M, Fu]

        # theta0(x_i): [N, M, 1] -> [N, M]
        theta0_output = self.theta0(xi_exp).squeeze(-1)  # [N, M]

        # theta1(x_i): [N, M, 1] -> [N, M]
        theta1_output = self.theta1(xi_exp).squeeze(-1)  # [N, M]

        # [N, M]
        price_exp = price.view(1, M).expand(N, M)        # [N, M]

        # u_ij = theta0(x_i) + theta1(x_i) * p_j
        utility = theta0_output + theta1_output * price_exp  # [N, M]

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output


In [288]:
# Apply the model
dml_model = UtilityEstimator_UserOnly(
    user_feature_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

dml_model.apply(init_weights)

UtilityEstimator_UserOnly(
  (theta0): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [289]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [290]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.8769532442092896, Validation Loss: 1.690571904182434
Epoch 2, Training Loss: 1.8308912515640259, Validation Loss: 1.6394579410552979
Epoch 3, Training Loss: 1.7913823127746582, Validation Loss: 1.6037553548812866
Epoch 4, Training Loss: 1.7591723203659058, Validation Loss: 1.5758402347564697
Epoch 5, Training Loss: 1.7330477237701416, Validation Loss: 1.5518667697906494
Epoch 6, Training Loss: 1.7097384929656982, Validation Loss: 1.5366288423538208
Epoch 7, Training Loss: 1.689908742904663, Validation Loss: 1.5263348817825317
Epoch 8, Training Loss: 1.6724510192871094, Validation Loss: 1.516879677772522
Epoch 9, Training Loss: 1.656850814819336, Validation Loss: 1.509648323059082
Epoch 10, Training Loss: 1.642682671546936, Validation Loss: 1.506511926651001
Epoch 11, Training Loss: 1.6305347681045532, Validation Loss: 1.504179835319519
Epoch 12, Training Loss: 1.6191965341567993, Validation Loss: 1.5008306503295898
Epoch 13, Training Loss: 1.6088275909423828, 

In [291]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [292]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon,  l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_user_only = abs((dedl - true) / true)
print(dedl_user_only)

-69.41780434515243
0.01110805342286285


In [293]:
# male
# Define Mask
male_mask = (X_user_test[:, 1] == 2)

# Call Function
res_male_user= evaluate_subgroup_dedl(
    model=dml_model,
    mask=male_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_male,                  # True value for Male
    discount_percentage=discount_percentage,
    total_n=300,                           # Total population size
    device=device
)

Subgroup N=38 | True Value=-80.6927
SDL: -40.8546 (MAPE: 0.4937)
DEDL: -82.1668 (MAPE: 0.0183)
------------------------------


In [294]:
# Define Mask
female_mask = (X_user_test[:, 1] == 1)

# Call Function
res_female_user = evaluate_subgroup_dedl(
    model=dml_model,
    mask=female_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_female,                # True value for Female
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=37 | True Value=-57.5479
SDL: -81.9233 (MAPE: 0.4236)
DEDL: -91.8506 (MAPE: 0.5961)
------------------------------


In [295]:
# phone usage less than 2 years
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)

res_phone_less_than_2_years_user= evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_less_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_less_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)


Subgroup N=7 | True Value=-93.1871
SDL: -168.1634 (MAPE: 0.8046)
DEDL: -180.7250 (MAPE: 0.9394)
------------------------------


In [296]:
# phone usage more than 2 years
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

res_phone_more_than_2_years_user = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_more_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_more_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=50 | True Value=-48.6022
SDL: -61.7253 (MAPE: 0.2700)
DEDL: -62.7838 (MAPE: 0.2918)
------------------------------


### Only product features

In [327]:
class UtilityEstimator_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_ProductOnly, self).__init__()

        # Process other products' features z_-j: input [M, (M-1)*Fp] -> output [M, Fp]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = product_feature_dim + product_feature_dim  # z_j + aggregated z_-j

        # theta0(z_j, aggregated z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(z_j, aggregated z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_product:        [M, Fp]
        x_other_products: [M, (M-1)*Fp]
        price:            [M]
        """
        device = x_product.device
        N = x_user.shape[0]
        M = x_product.shape[0]
        Fp = x_product.shape[1]

        # Aggregate z_-j: [M, (M-1)*Fp] -> [M, Fp]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, Fp]

        # Expand to [N, M, *]
        zj_exp     = x_product.unsqueeze(0).expand(N, -1, -1)                 # [N, M, Fp]
        zminus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1) # [N, M, Fp]

        feat_theta = torch.cat([zj_exp, zminus_exp], dim=2)                   # [N, M, 2*Fp]

        theta0_output = self.theta0(feat_theta).squeeze(-1)

        theta1_output = self.theta1(feat_theta).squeeze(-1)

        # Broadcast prices to [N, M]
        price_exp = price.view(1, M).expand(N, M)                             # [N, M]

        # Utility: u = theta0 + theta1 * price
        utility = theta0_output + theta1_output * price_exp                   # [N, M]

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [328]:
# Apply the model
dml_model = UtilityEstimator_ProductOnly(
    product_feature_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=3
).to(device)

# Initialize weight
dml_model.apply(init_weights)

UtilityEstimator_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
  )
  (theta0): Sequential(
    (0): Linear(in_features=6, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=6, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [329]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [330]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 3.4414072036743164, Validation Loss: 3.2331864833831787
Epoch 2, Training Loss: 3.103393793106079, Validation Loss: 2.877876043319702
Epoch 3, Training Loss: 2.739011526107788, Validation Loss: 2.528287649154663
Epoch 4, Training Loss: 2.381101608276367, Validation Loss: 2.200990676879883
Epoch 5, Training Loss: 2.0435633659362793, Validation Loss: 1.8934599161148071
Epoch 6, Training Loss: 1.7469710111618042, Validation Loss: 1.6553866863250732
Epoch 7, Training Loss: 1.5219515562057495, Validation Loss: 1.5180760622024536
Epoch 8, Training Loss: 1.3856451511383057, Validation Loss: 1.4659074544906616
Epoch 9, Training Loss: 1.3371837139129639, Validation Loss: 1.4423421621322632
Epoch 10, Training Loss: 1.346558928489685, Validation Loss: 1.4494526386260986
Epoch 11, Training Loss: 1.387393832206726, Validation Loss: 1.4843131303787231
Epoch 12, Training Loss: 1.4289394617080688, Validation Loss: 1.509865403175354
Epoch 13, Training Loss: 1.4563874006271362, V

In [331]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [332]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_product_only = abs((dedl - true) / true)
print(dedl_product_only)

-67.53208475901668
0.016358477361645175


In [333]:
# Define Mask
male_mask = (X_user_test[:, 1] == 2)

# Call Function
res_male_prod = evaluate_subgroup_dedl(
    model=dml_model,
    mask=male_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_male,                  # True value for Male
    discount_percentage=discount_percentage,
    total_n=300,                           # Total population size
    device=device
)

Subgroup N=38 | True Value=-80.6927
SDL: -35.2603 (MAPE: 0.5630)
DEDL: -82.4707 (MAPE: 0.0220)
------------------------------


In [334]:
# Define Mask
female_mask = (X_user_test[:, 1] == 1)

# Call Function
res_female_prod = evaluate_subgroup_dedl(
    model=dml_model,
    mask=female_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_female,                # True value for Female
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=37 | True Value=-57.5479
SDL: -35.2603 (MAPE: 0.3873)
DEDL: -56.0382 (MAPE: 0.0262)
------------------------------


In [335]:
# phone usage less than 2 years
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)

res_phone_less_than_2_years_prod = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_less_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_less_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=7 | True Value=-93.1871
SDL: -35.2603 (MAPE: 0.6216)
DEDL: -91.9578 (MAPE: 0.0132)
------------------------------


In [336]:
# phone usage more than 2 years
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

res_phone_more_than_2_years_prod = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_more_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_more_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=50 | True Value=-48.6022
SDL: -35.2603 (MAPE: 0.2745)
DEDL: -46.8241 (MAPE: 0.0366)
------------------------------


### All features

In [337]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim, num_product, hidden=16, dropout=0.1): 
        super(UtilityEstimator, self).__init__()

        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 8), 
            nn.ReLU(),
            nn.Linear(8, product_feature_dim)
        )

        in_dim = user_feature_dim + 2 * product_feature_dim  # x_i, z_j, z_-j

        # theta0(x_i, z_j, z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1) 
        )

        # theta1(x_i, z_j, z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        N = x_user.shape[0]
        M = x_product.shape[0]

        aggregated_other_features = self.other_product_features_layers(x_other_products)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)
        z_j_exp    = x_product.unsqueeze(0).expand(N, -1, -1)
        z_minus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1)

        combined_features = torch.cat(
            (x_user_exp, z_j_exp, z_minus_exp),
            dim=2
        )

        theta0_output = self.theta0(combined_features).squeeze(-1)
        theta1_output = self.theta1(combined_features).squeeze(-1)

        utility = theta0_output + theta1_output * price

        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)

        return utilities_with_outside, theta0_output, theta1_output

        return utilities_with_outside, theta0_output, theta1_output

In [338]:
# Apply the model
# Apply the model
dml_model = UtilityEstimator(
    user_feature_dim=X_user.shape[1],
    product_feature_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=64,
    dropout=0.1
).to(device)
dml_model.apply(init_weights)

UtilityEstimator(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=3, bias=True)
  )
  (theta0): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [339]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [340]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.3250224590301514, Validation Loss: 1.6894055604934692
Epoch 2, Training Loss: 1.7268120050430298, Validation Loss: 1.6822025775909424
Epoch 3, Training Loss: 1.71754789352417, Validation Loss: 1.2865389585494995
Epoch 4, Training Loss: 1.3194303512573242, Validation Loss: 1.1941434144973755
Epoch 5, Training Loss: 1.3312824964523315, Validation Loss: 1.1698169708251953
Epoch 6, Training Loss: 1.2739955186843872, Validation Loss: 1.2087175846099854
Epoch 7, Training Loss: 1.248382806777954, Validation Loss: 1.2455586194992065
Epoch 8, Training Loss: 1.2920658588409424, Validation Loss: 1.1058964729309082
Epoch 9, Training Loss: 1.1939436197280884, Validation Loss: 1.0930336713790894
Epoch 10, Training Loss: 1.2257577180862427, Validation Loss: 1.100737452507019
Epoch 11, Training Loss: 1.2298120260238647, Validation Loss: 1.0736688375473022
Epoch 12, Training Loss: 1.175995945930481, Validation Loss: 1.1222971677780151
Epoch 13, Training Loss: 1.196056962013244

In [341]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [342]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
0.1,1, ]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_all = abs((dedl - true) / true)
print(dedl_all)

-69.02091807888863
0.0053271892209790575


In [343]:
# Define Mask
male_mask = (X_user_test[:, 1] == 2)

# Call Function
res_male_all = evaluate_subgroup_dedl(
    model=dml_model,
    mask=male_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_male,                  # True value for Male
    discount_percentage=discount_percentage,
    total_n=300,                           # Total population size
    device=device
)

Subgroup N=38 | True Value=-80.6927
SDL: -41.7750 (MAPE: 0.4823)
DEDL: -68.7488 (MAPE: 0.1480)
------------------------------


In [344]:
# Define Mask
female_mask = (X_user_test[:, 1] == 1)

# Call Function
res_female_all = evaluate_subgroup_dedl(
    model=dml_model,
    mask=female_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization, # Passed as is
    true_value=true_female,                # True value for Female
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=37 | True Value=-57.5479
SDL: -54.8882 (MAPE: 0.0462)
DEDL: -62.0106 (MAPE: 0.0775)
------------------------------


In [345]:
# phone usage less than 2 years
phone_less_than_2_years_mask = (X_user_test[:, 2] == 1) | (X_user_test[:, 2] == 5)

res_phone_less_than_2_years_all = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_less_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_less_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)


Subgroup N=7 | True Value=-93.1871
SDL: -112.2915 (MAPE: 0.2050)
DEDL: -92.0372 (MAPE: 0.0123)
------------------------------


In [346]:
# phone usage more than 2 years
phone_more_than_2_years_mask = (X_user_test[:, 2] == 2) | (X_user_test[:, 2] == 3) | (X_user_test[:, 2] == 6)

res_phone_more_than_2_years_all = evaluate_subgroup_dedl(
    model=dml_model,
    mask=phone_more_than_2_years_mask,
    X_user_test=X_user_test,
    X_product=X_product,
    choice_test=choice_test,
    price=price,
    prod_randomization=prod_randomization,
    true_value=true_phone_more_than_2_years,
    discount_percentage=discount_percentage,
    total_n=300,
    device=device
)

Subgroup N=50 | True Value=-48.6022
SDL: -53.5173 (MAPE: 0.1011)
DEDL: -54.3693 (MAPE: 0.1187)
------------------------------


# Results

In [349]:
# Collect all APE values (overall)
ape_results = []

# Naive
try:
    ape_results.append({'Method': 'Naive', 'Subgroup': 'Overall', 'APE': naive_pe})
except NameError:
    pass

# Linear methods
try:
    ape_results.append({'Method': 'Linear (No features)', 'Subgroup': 'Overall', 'APE': linear_no})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (User only)', 'Subgroup': 'Overall', 'APE': linear_user_only})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (Product only)', 'Subgroup': 'Overall', 'APE': linear_product_only})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (All features)', 'Subgroup': 'Overall', 'APE': linear_all})
except NameError:
    pass

# PDL methods
try:
    ape_results.append({'Method': 'PDL (No features)', 'Subgroup': 'Overall', 'APE': pdl_no})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (User only)', 'Subgroup': 'Overall', 'APE': pdl_user})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (Product only)', 'Subgroup': 'Overall', 'APE': pdl_product})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (All features)', 'Subgroup': 'Overall', 'APE': pdl_all})
except NameError:
    pass

# DEDL methods
try:
    ape_results.append({'Method': 'DEDL (No features)', 'Subgroup': 'Overall', 'APE': dedl_no})
except NameError:
    pass

try:
    ape_results.append({'Method': 'DEDL (User only)', 'Subgroup': 'Overall', 'APE': dedl_user_only})
except NameError:
    pass

try:
    ape_results.append({'Method': 'DEDL (Product only)', 'Subgroup': 'Overall', 'APE': dedl_product_only})
except NameError:
    pass

try:
    ape_results.append({'Method': 'DEDL (All features)', 'Subgroup': 'Overall', 'APE': dedl_all})
except NameError:
    pass

# === Subgroup APEs: Male ===
# Naive
try:
    ape_results.append({'Method': 'Naive', 'Subgroup': 'Male', 'APE': naive_m})
except NameError:
    pass

# Linear methods
try:
    ape_results.append({'Method': 'Linear (No features)', 'Subgroup': 'Male', 'APE': linear_no_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (User only)', 'Subgroup': 'Male', 'APE': linear_user_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (Product only)', 'Subgroup': 'Male', 'APE': linear_prod_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (All features)', 'Subgroup': 'Male', 'APE': linear_all_m})
except NameError:
    pass

# PDL methods
try:
    ape_results.append({'Method': 'PDL (No features)', 'Subgroup': 'Male', 'APE': pdl_no_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (User only)', 'Subgroup': 'Male', 'APE': pdl_user_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (Product only)', 'Subgroup': 'Male', 'APE': pdl_prod_m})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (All features)', 'Subgroup': 'Male', 'APE': pdl_all_m})
except NameError:
    pass

# DEDL methods (from evaluate_subgroup_dedl results)
try:
    if 'res_male' in globals() and res_male is not None:
        ape_results.append({'Method': 'DEDL (No features)', 'Subgroup': 'Male', 'APE': res_male['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_male_user' in globals() and res_male_user is not None:
        ape_results.append({'Method': 'DEDL (User only)', 'Subgroup': 'Male', 'APE': res_male_user['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_male_prod' in globals() and res_male_prod is not None:
        ape_results.append({'Method': 'DEDL (Product only)', 'Subgroup': 'Male', 'APE': res_male_prod['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_male_all' in globals() and res_male_all is not None:
        ape_results.append({'Method': 'DEDL (All features)', 'Subgroup': 'Male', 'APE': res_male_all['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

# === Subgroup APEs: Female ===
# Naive
try:
    ape_results.append({'Method': 'Naive', 'Subgroup': 'Female', 'APE': naive_f})
except NameError:
    pass

# Linear methods
try:
    ape_results.append({'Method': 'Linear (No features)', 'Subgroup': 'Female', 'APE': linear_no_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (User only)', 'Subgroup': 'Female', 'APE': linear_user_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (Product only)', 'Subgroup': 'Female', 'APE': linear_prod_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (All features)', 'Subgroup': 'Female', 'APE': linear_all_f})
except NameError:
    pass

# PDL methods
try:
    ape_results.append({'Method': 'PDL (No features)', 'Subgroup': 'Female', 'APE': pdl_no_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (User only)', 'Subgroup': 'Female', 'APE': pdl_user_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (Product only)', 'Subgroup': 'Female', 'APE': pdl_prod_f})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (All features)', 'Subgroup': 'Female', 'APE': pdl_all_f})
except NameError:
    pass

# DEDL methods
try:
    if 'res_female' in globals() and res_female is not None:
        ape_results.append({'Method': 'DEDL (No features)', 'Subgroup': 'Female', 'APE': res_female['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_female_user' in globals() and res_female_user is not None:
        ape_results.append({'Method': 'DEDL (User only)', 'Subgroup': 'Female', 'APE': res_female_user['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_female_prod' in globals() and res_female_prod is not None:
        ape_results.append({'Method': 'DEDL (Product only)', 'Subgroup': 'Female', 'APE': res_female_prod['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_female_all' in globals() and res_female_all is not None:
        ape_results.append({'Method': 'DEDL (All features)', 'Subgroup': 'Female', 'APE': res_female_all['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

# === Subgroup APEs: Phone Usage < 2 years ===
# Naive
try:
    ape_results.append({'Method': 'Naive', 'Subgroup': 'Phone < 2 years', 'APE': naive_phone_less_than_2_years})
except NameError:
    pass

# Linear methods
try:
    ape_results.append({'Method': 'Linear (No features)', 'Subgroup': 'Phone < 2 years', 'APE': linear_no_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (User only)', 'Subgroup': 'Phone < 2 years', 'APE': linear_user_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (Product only)', 'Subgroup': 'Phone < 2 years', 'APE': linear_prod_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (All features)', 'Subgroup': 'Phone < 2 years', 'APE': linear_all_phone_less_than_2_years})
except NameError:
    pass

# PDL methods
try:
    ape_results.append({'Method': 'PDL (No features)', 'Subgroup': 'Phone < 2 years', 'APE': pdl_no_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (User only)', 'Subgroup': 'Phone < 2 years', 'APE': pdl_user_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (Product only)', 'Subgroup': 'Phone < 2 years', 'APE': pdl_prod_phone_less_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (All features)', 'Subgroup': 'Phone < 2 years', 'APE': pdl_all_phone_less_than_2_years})
except NameError:
    pass

# DEDL methods
try:
    if 'res_phone_less_than_2_years' in globals() and res_phone_less_than_2_years is not None:
        ape_results.append({'Method': 'DEDL (No features)', 'Subgroup': 'Phone < 2 years', 'APE': res_phone_less_than_2_years['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_less_than_2_years_user' in globals() and res_phone_less_than_2_years_user is not None:
        ape_results.append({'Method': 'DEDL (User only)', 'Subgroup': 'Phone < 2 years', 'APE': res_phone_less_than_2_years_user['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_less_than_2_years_prod' in globals() and res_phone_less_than_2_years_prod is not None:
        ape_results.append({'Method': 'DEDL (Product only)', 'Subgroup': 'Phone < 2 years', 'APE': res_phone_less_than_2_years_prod['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_less_than_2_years_all' in globals() and res_phone_less_than_2_years_all is not None:
        ape_results.append({'Method': 'DEDL (All features)', 'Subgroup': 'Phone < 2 years', 'APE': res_phone_less_than_2_years_all['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

# === Subgroup APEs: Phone Usage >= 2 years ===
# Naive
try:
    ape_results.append({'Method': 'Naive', 'Subgroup': 'Phone >= 2 years', 'APE': naive_phone_more_than_2_years})
except NameError:
    pass

# Linear methods
try:
    ape_results.append({'Method': 'Linear (No features)', 'Subgroup': 'Phone >= 2 years', 'APE': linear_no_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (User only)', 'Subgroup': 'Phone >= 2 years', 'APE': linear_user_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (Product only)', 'Subgroup': 'Phone >= 2 years', 'APE': linear_prod_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'Linear (All features)', 'Subgroup': 'Phone >= 2 years', 'APE': linear_all_phone_more_than_2_years})
except NameError:
    pass

# PDL methods
try:
    ape_results.append({'Method': 'PDL (No features)', 'Subgroup': 'Phone >= 2 years', 'APE': pdl_no_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (User only)', 'Subgroup': 'Phone >= 2 years', 'APE': pdl_user_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (Product only)', 'Subgroup': 'Phone >= 2 years', 'APE': pdl_prod_phone_more_than_2_years})
except NameError:
    pass

try:
    ape_results.append({'Method': 'PDL (All features)', 'Subgroup': 'Phone >= 2 years', 'APE': pdl_all_phone_more_than_2_years})
except NameError:
    pass

# DEDL methods
try:
    if 'res_phone_more_than_2_years' in globals() and res_phone_more_than_2_years is not None:
        ape_results.append({'Method': 'DEDL (No features)', 'Subgroup': 'Phone >= 2 years', 'APE': res_phone_more_than_2_years['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_more_than_2_years_user' in globals() and res_phone_more_than_2_years_user is not None:
        ape_results.append({'Method': 'DEDL (User only)', 'Subgroup': 'Phone >= 2 years', 'APE': res_phone_more_than_2_years_user['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_more_than_2_years_prod' in globals() and res_phone_more_than_2_years_prod is not None:
        ape_results.append({'Method': 'DEDL (Product only)', 'Subgroup': 'Phone >= 2 years', 'APE': res_phone_more_than_2_years_prod['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

try:
    if 'res_phone_more_than_2_years_all' in globals() and res_phone_more_than_2_years_all is not None:
        ape_results.append({'Method': 'DEDL (All features)', 'Subgroup': 'Phone >= 2 years', 'APE': res_phone_more_than_2_years_all['dedl_mape']})
except (NameError, KeyError, TypeError):
    pass

# Create DataFrame
if ape_results:
    ape_df = pd.DataFrame(ape_results)
    ape_df['APE (%)'] = (ape_df['APE'] * 100).round(4)
    
    # Display the table
    print("=" * 100)
    print("Summary of All Methods - Absolute Percentage Error (APE) by Subgroup")
    print("=" * 100)
    print(ape_df[['Method', 'Subgroup', 'APE (%)']].to_string(index=False))
    print("=" * 100)
    
    # Also create a pivot table for better visualization
    print("\n")
    print("=" * 100)
    print("APE (%) Pivot Table")
    print("=" * 100)
    pivot_df = ape_df.pivot_table(index='Method', columns='Subgroup', values='APE (%)', aggfunc='first')
    print(pivot_df.to_string())
    print("=" * 100)
    
    # Return the DataFrame
    ape_df
else:
    print("No APE values found. Make sure all models have been evaluated.")


Summary of All Methods - Absolute Percentage Error (APE) by Subgroup
               Method         Subgroup  APE (%)
                Naive          Overall  11.0248
 Linear (No features)          Overall   9.8224
   Linear (User only)          Overall   9.7069
Linear (Product only)          Overall  30.2494
Linear (All features)          Overall  27.7544
    PDL (No features)          Overall   8.2023
      PDL (User only)          Overall   4.6603
   PDL (Product only)          Overall   7.5108
   PDL (All features)          Overall   3.3029
   DEDL (No features)          Overall   2.5995
     DEDL (User only)          Overall   1.1108
  DEDL (Product only)          Overall   1.6358
  DEDL (All features)          Overall   0.5327
                Naive             Male  34.0103
 Linear (No features)             Male  23.2748
   Linear (User only)             Male  12.5845
Linear (Product only)             Male  40.6546
Linear (All features)             Male  29.5230
    PDL (No feature